# Test Finite Differences Poisson

In [1]:
%matplotlib inline
import numpy as np
from scipy import linalg
from scipy import sparse
from scipy.sparse import linalg as la
import matplotlib.pyplot as plt

# Normal Finite Differences kernel on a 2D grid...

1. 

In [2]:
Nx = 201
Ny = 201
x = np.linspace(0, 1, Nx)
y = np.linspace(0, 1, Ny)
X, Y = np.meshgrid(x, y)
dX = np.diff(X, axis=1) # Columns
dY = np.diff(Y, axis=0) # Rows
hx = dX.mean()
hy = dY.mean()

In [3]:
X.ravel() # Fast changing index...


array([0.   , 0.005, 0.01 , ..., 0.99 , 0.995, 1.   ])

In [4]:
hy

0.005000000000000001

In [5]:
dY

array([[0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005],
       [0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005],
       [0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005],
       ...,
       [0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005],
       [0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005],
       [0.005, 0.005, 0.005, ..., 0.005, 0.005, 0.005]])

In [6]:
Phi_exact = X**3 + 2 * Y**3
source = -6 * X + -12 * Y
f0 = source.ravel()

In [9]:
def boundary(Nx, Ny):
    bc = np.zeros(Nx*Ny,dtype=bool)
    for i in range(Ny):
        for j in range(Nx):
            if i == 0 or j == 0 or i == (Ny-1) or j == (Nx-1):
                ind = i * Nx + j # This point
                bc[ind] = True
    return bc

In [10]:
def poisson_equally_spaced(Nx, Ny, hx, hy):
    A = sparse.lil_matrix((Nx*Ny, Nx*Ny))
    for i in range(Ny):
        for j in range(Nx):
            ind = i * Nx + j # This point
            ixp = ind + 1    # +x 
            ixn = ind - 1    # -x
            iyp = (i+1)*Nx + j  # +y
            iyn = (i-1)*Nx + j  # -y

            A[ind, ind] = 2 / hx**2 + 2 / hy**2
            if j < (Nx - 1):
                A[ind, ixp] = -1 / hx**2
            if j > 0:
                A[ind, ixn] = -1 / hx**2

            if i > 0:
                A[ind, iyn] = -1 / hy**2
            if i < (Ny-1):
                A[ind, iyp] = -1 / hy**2
    
    return sparse.csr_matrix(A) # Convert to better format for usage
        

In [67]:
def poisson_variable_spacing(x, y):
    Nx = len(x)
    Ny = len(y)
    hx = np.diff(x)
    hy = np.diff(y)
    A = sparse.lil_matrix((Nx*Ny, Nx*Ny))
    for i in range(Ny):
        for j in range(Nx):
            ind = i * Nx + j # This point
            ixp = ind + 1    # +x 
            ixn = ind - 1    # -x
            iyp = (i+1)*Nx + j  # +y
            iyn = (i-1)*Nx + j  # -y
            
            
            Dx_plus = hx[j] if j < (Nx-1) else 0.0
            Dx_minus = hx[j-1] if j > 0 else 0.0
            Dy_plus = hy[i] if i < (Ny-1) else 0.0
            Dy_minus = hy[i-1] if i > 0 else 0.0
            
            prefactor_x = 4/((Dx_plus+Dx_minus)*(Dx_plus**2 + Dx_minus**2))
            
            prefactor_y = 4/((Dy_plus+Dy_minus)*(Dy_plus**2 + Dy_minus**2))

            
            A[ind, ind] = (Dx_plus+Dx_minus) * prefactor_x + (Dy_plus+Dy_minus) * prefactor_y
            if j < (Nx - 1):
                A[ind, ixp] = -1 * Dx_minus * prefactor_x
            if j > 0:
                A[ind, ixn] = -1 * Dx_plus * prefactor_x

            if i > 0:
                A[ind, iyn] = -1 * Dy_plus * prefactor_y
            if i < (Ny-1):
                A[ind, iyp] = -1 * Dy_minus * prefactor_y
    
    return sparse.csr_matrix(A) # Convert to better format for usage

In [68]:
A = poisson_equally_spaced(Nx, Ny, hx, hy)
bc = boundary(Nx, Ny)

In [69]:
bc.reshape((Ny, Nx))

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, Fals

In [70]:
u = np.zeros(Nx*Ny)
u[bc] = Phi_exact.ravel()[bc]

In [71]:
u.reshape((Ny, Nx))

array([[0.000000e+00, 1.000000e-03, 8.000000e-03, 2.700000e-02,
        6.400000e-02, 1.250000e-01, 1.663750e-01, 2.160000e-01,
        2.746250e-01, 3.430000e-01, 4.218750e-01, 5.120000e-01,
        6.141250e-01, 7.290000e-01, 8.573750e-01, 1.000000e+00],
       [2.000000e-03, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.002000e+00],
       [1.600000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.016000e+00],
       [5.400000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0

In [72]:
f = f0 - A @ u
A_cut = A[~bc].T[~bc].T # 
f_cut = f[~bc]

In [73]:
u_cut = la.spsolve(A_cut, f_cut)
u[~bc] = u_cut

In [74]:
%time la.spsolve(A_cut, f_cut)

CPU times: user 1.36 ms, sys: 1.09 ms, total: 2.45 ms
Wall time: 1.31 ms


array([0.00548962, 0.01039413, 0.02379256, 0.0498567 , 0.08882132,
       0.12935344, 0.17734592, 0.23498445, 0.30354797, 0.38409681,
       0.47766367, 0.58530587, 0.70807872, 0.84686906, 0.01510607,
       0.01447983, 0.02110489, 0.03812653, 0.06595515, 0.10209459,
       0.14779092, 0.20448535, 0.27344993, 0.35590667, 0.45313027,
       0.56650612, 0.69752446, 0.84751662, 0.03489473, 0.02537679,
       0.02485589, 0.0345916 , 0.05530374, 0.08711843, 0.13043462,
       0.18623268, 0.25568499, 0.34009526, 0.44093508, 0.55992245,
       0.6991878 , 0.86179558, 0.06175207, 0.04587814, 0.03930676,
       0.04327225, 0.05884583, 0.08706325, 0.12827411, 0.18330109,
       0.25326419, 0.33952038, 0.44367691, 0.56765522, 0.71381984,
       0.8852121 , 0.0988935 , 0.07782216, 0.06631541, 0.06575285,
       0.07745761, 0.10284361, 0.14234673, 0.19676871, 0.2672242 ,
       0.35511305, 0.46212843, 0.59029484, 0.74202897, 0.92019343,
       0.14846432, 0.12308093, 0.10764087, 0.10359625, 0.11240

In [75]:
u.reshape((Ny, Nx))

array([[0.00000000e+00, 1.00000000e-03, 8.00000000e-03, 2.70000000e-02,
        6.40000000e-02, 1.25000000e-01, 1.66375000e-01, 2.16000000e-01,
        2.74625000e-01, 3.43000000e-01, 4.21875000e-01, 5.12000000e-01,
        6.14125000e-01, 7.29000000e-01, 8.57375000e-01, 1.00000000e+00],
       [2.00000000e-03, 5.48962263e-03, 1.03941252e-02, 2.37925587e-02,
        4.98567016e-02, 8.88213232e-02, 1.29353443e-01, 1.77345915e-01,
        2.34984451e-01, 3.03547971e-01, 3.84096810e-01, 4.77663667e-01,
        5.85305868e-01, 7.08078721e-01, 8.46869056e-01, 1.00200000e+00],
       [1.60000000e-02, 1.51060688e-02, 1.44798268e-02, 2.11048939e-02,
        3.81265254e-02, 6.59551480e-02, 1.02094587e-01, 1.47790916e-01,
        2.04485351e-01, 2.73449931e-01, 3.55906667e-01, 4.53130267e-01,
        5.66506124e-01, 6.97524455e-01, 8.47516616e-01, 1.01600000e+00],
       [5.40000000e-02, 3.48947292e-02, 2.53767929e-02, 2.48558899e-02,
        3.45915983e-02, 5.53037351e-02, 8.71184289e-02, 1.304

In [76]:
Phi_exact

array([[0.000000e+00, 1.000000e-03, 8.000000e-03, 2.700000e-02,
        6.400000e-02, 1.250000e-01, 1.663750e-01, 2.160000e-01,
        2.746250e-01, 3.430000e-01, 4.218750e-01, 5.120000e-01,
        6.141250e-01, 7.290000e-01, 8.573750e-01, 1.000000e+00],
       [2.000000e-03, 3.000000e-03, 1.000000e-02, 2.900000e-02,
        6.600000e-02, 1.270000e-01, 1.683750e-01, 2.180000e-01,
        2.766250e-01, 3.450000e-01, 4.238750e-01, 5.140000e-01,
        6.161250e-01, 7.310000e-01, 8.593750e-01, 1.002000e+00],
       [1.600000e-02, 1.700000e-02, 2.400000e-02, 4.300000e-02,
        8.000000e-02, 1.410000e-01, 1.823750e-01, 2.320000e-01,
        2.906250e-01, 3.590000e-01, 4.378750e-01, 5.280000e-01,
        6.301250e-01, 7.450000e-01, 8.733750e-01, 1.016000e+00],
       [5.400000e-02, 5.500000e-02, 6.200000e-02, 8.100000e-02,
        1.180000e-01, 1.790000e-01, 2.203750e-01, 2.700000e-01,
        3.286250e-01, 3.970000e-01, 4.758750e-01, 5.660000e-01,
        6.681250e-01, 7.830000e-01, 9

In [77]:
abs(u.reshape((Ny, Nx)) - Phi_exact).mean()

0.10257119970728046

# 

- For a 101x101 matrix, it takes 1.0 s system time (6.73 s wall time)
- Using sparse matrices, it takes 79 ms wall time (
- Using a 201x201 matrix, it takes only 379 ms wall time (just over 4x the time...)

# Variable grid test

In [140]:
ddx = list(reversed([0.0105 * 1.05**i for i in range(51)]))
x = np.cumsum(ddx)
ddy = list(reversed([0.011 * 1.023**i for i in range(51)]))
ddy
y = np.cumsum(ddy)

In [141]:
y

array([0.03429054, 0.06781014, 0.10057611, 0.13260541, 0.1639146 ,
       0.19451987, 0.22443705, 0.2536816 , 0.28226864, 0.31021297,
       0.33752903, 0.36423094, 0.39033252, 0.41584726, 0.44078836,
       0.4651687 , 0.48900091, 0.5122973 , 0.53506991, 0.55733054,
       0.57909068, 0.60036159, 0.62115427, 0.64147946, 0.66134769,
       0.68076923, 0.69975411, 0.71831216, 0.73645296, 0.75418591,
       0.77152018, 0.78846471, 0.80502829, 0.82121947, 0.83704662,
       0.85251794, 0.86764141, 0.88242486, 0.89687594, 0.91100212,
       0.9248107 , 0.93830882, 0.95150347, 0.96440146, 0.97700947,
       0.98933401, 1.00138147, 1.01315806, 1.02466988, 1.03592288,
       1.04692288])

In [142]:

Nx, Ny = len(x), len(y)

In [143]:
X, Y = np.meshgrid(x, y)
dX = np.diff(X, axis=1) # Columns
dY = np.diff(Y, axis=0) # Rows
hx = dX.mean()
hy = dY.mean()
Phi_exact = X**3 + 2 * Y**3
source = -6 * X + -12 * Y
f0 = source.ravel()

In [144]:
A = poisson_variable_spacing(x, y)
bc = boundary(len(x), len(y))

In [145]:
u = np.zeros(Nx*Ny)
u[bc] = Phi_exact.ravel()[bc]

In [146]:
# Add some functions to do this...
f = f0 - A @ u
A_cut = A[~bc].T[~bc].T # 
f_cut = f[~bc]

In [147]:
u_cut = la.spsolve(A_cut, f_cut)
u[~bc] = u_cut

In [148]:
abs(u.reshape((Ny, Nx)) - Phi_exact).mean()

0.0009280511822999454

# Seems to be a working variable grid solver!

(Note: Only works for constant varepsilon...

In [157]:
A[0:6, 0:6].toarray()

array([[3864.2822242 ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [ -77.80702622, 3719.60702805,  -81.69737754,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -85.78224641, 3735.95622943,  -90.07135873,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -94.57492667, 3753.98122396,
         -99.303673  ,    0.        ],
       [   0.        ,    0.        ,    0.        , -104.26885665,
        3773.85378043, -109.48229949],
       [   0.        ,    0.        ,    0.        ,    0.        ,
        -114.95641446, 3795.76327393]])

In [161]:
f_cut[:32]

array([   9.72383315,   33.92065198,   77.65178712,  143.55782346,
        233.21110775,  347.19980852,  485.34986151,  646.90774066,
        830.69025833, 1035.20667095, 1258.75757308, 1499.5143848 ,
       1755.58265762, 2025.05192941, 2306.03443678, 2596.69463313,
       2895.27115413, 3200.09261153, 3509.58837385, 3822.29530468,
       4136.86126869, 4452.04608052, 4766.72045678, 5079.8634344 ,
       5390.5586372 , 5697.98970316, 6001.43512769, 6300.26272893,
       6593.92390123, 6881.94778846, 7163.93548112, 7439.55431755])